# Bandgap Workflow Example
 This notebook demonstrates how to build and run a bandgap workflow for a material.
 Example of building and running a bandgap workflow for twisted MoS2 interface from specific_examples.

## Process Overview
### 1. Set up the environment and parameters.
### 2. Log in to get the API token
### 3. Load the target material.
### 4. Import workflow builder and related analyzers.
### 5. Analyze material to get parameters for the workflow configuration.
### 6. Create the workflow configuration.
### 7. Create a job with material and workflow configuration.
### 8. Submit the job to the server.
### 9. Monitor the job status and retrieve results.
### 10. Display the results.

## 1. Set up the environment and parameters

## 2. Log in to get the API token

In [ ]:
from mat3ra.api import ApiClient

# Log in to get the API token
auth_config = await ApiClient().login()

## 3. Load the target material

In [ ]:
from utils.visualize import visualize_materials as visualize
from utils.jupyterlite import load_material_from_folder

material = load_material_from_folder("/uploads", "MoS2_twisted_interface_60_degrees.json")
visualize(material)

## 5. Create workflow and set its parameters
### 5.1. Get list of applications and select one

In [ ]:
from mat3ra.standata.applications import Applications

# Get Applications list (with versions, build)
apps_list = Applications.list_all()
# returns apps_list[0] = [{"name" : "espresso", "version": "7.2", "build": "GNU"}]

app = Applications.get_by_name_first_match("espresso")
# returns name, version, build config

### 5.2. Create workflow from standard workflows and preview it

In [ ]:
from mat3ra.standata.workflows import Workflows
from mat3ra.wode.workflows import Workflow
from utils.visualize import visualize_workflow

# Search WF by name and application
workflow_config = Workflows.filter_by_application(app).get_by_name_first_match("band_gap")
workflow = Workflow.create(workflow_config)

# View workflow to understand its structure
visualize_workflow(workflow)

### 5.3. Add relaxation subworkflow

In [ ]:
workflow.add_relaxation()
visualize_workflow(workflow)
# Relaxation subworkflow is added as the first subworkflow

### 5.4. Change subworkflow details (Model subtype)

In [ ]:
from mat3ra.standata.model_tree import ModelTreeStandata

swf_0 = workflow.subworkflows[0]  # relaxation subworkflow
swf_1 = workflow.subworkflows[1]  # band structure subworkflow

# Change model subtype for relaxation subworkflow
# For preview:
subtypes = ModelTreeStandata.get_subtypes_by_model_type("dft")  # ["gga", "lda"] as enum
functionals = ModelTreeStandata.get_functionals_by_subtype("dft", "lda")  # ["pz", ...] as enum

model = ModelTreeStandata.get_model_by_parameters(
    model_type="dft",
    subtype=subtypes.LDA,
    functional=functionals.PZ,
)
swf_0.model = model
swf_1.model = model

### 5.5. Modify workflow units found in preview

In [ ]:
from mat3ra.wode.context_providers import PointsGridFormDataProvider

# Values from publication
kgrid_scf = [6, 6, 1]
kgrid_nscf = [12, 12, 1]
kgrid_relax = kgrid_scf

kgrid_context_provider = PointsGridFormDataProvider(material=material)

new_context_relax = kgrid_context_provider.get_data(dimensions=kgrid_relax)
new_context_scf = kgrid_context_provider.get_data(dimensions=kgrid_scf)
new_context_nscf = kgrid_context_provider.get_data(dimensions=kgrid_nscf)

# Get workflow's specific unit that needs to be modified
# Option 1: search is done by unit name regex across the entire workflow
unit_to_modify_relax = workflow.get_unit_by_name(name_regex="relax")
unit_to_modify_relax.context.add_context(new_context_relax)

# Option 2: search is done by unit name within a specific subworkflow
unit_to_modify_scf = workflow.subworkflows[1].get_unit_by_name(name="pw_scf")
unit_to_modify_scf.context.add_context(new_context_scf)
unit_to_modify_nscf = workflow.subworkflows[1].get_unit_by_name(name="pw_nscf")
unit_to_modify_nscf.context.add_context(new_context_nscf)

# Set the modified unit back to the workflow
# Option 1: direct set by unit object, replacing the existing one
workflow.set_unit(unit_to_modify_relax)

# Option 2: set by unit flowchart id and new unit object
workflow.set_unit(unit_flowchart_id=unit_to_modify_scf.flowchart_id, new_unit=unit_to_modify_scf)
workflow.set_unit(unit_flowchart_id=unit_to_modify_nscf.flowchart_id, new_unit=unit_to_modify_nscf)

## 6. Create the compute configuration
### 6.1. View available clusters and providers

In [ ]:
# List available compute providers and clusters
from mat3ra.ide.compute import ComputeProvider, ComputeCluster

providers = ComputeProvider.list_all()
clusters = ComputeCluster.list_all()

### 6.2. Create compute configuration

In [ ]:
from mat3ra.ide.compute import ComputeConfiguration, QueueTypesEnum

compute_config = ComputeConfiguration(
    queue=QueueTypesEnum.OR8,
    nodes=1,
    ppn=8,
    cluster=clusters[0],  # select first available cluster
)

## 7. Create the job with material and workflow configuration

In [ ]:
from mat3ra.jode.job import create_job

job = create_job(
    workflow=workflow,
    material=material,
    compute=compute_config,
    auth_config=auth_config
)


## 8. Submit the job and monitor the status

In [ ]:
from mat3ra.prode import PropertyEnum

job.run()
job.wait_for_complete()
# job.check_status()
# job.get_current_output()

## 9. Retrieve results

In [ ]:
# AFTER Finished
# A class from Prode to handle results
results = job.get_results(PropertyEnum.BAND_GAP, PropertyEnum.BAND_STRUCTURE)

## 10. Display results

In [ ]:
# Visual library that can visualize any property defined in Prode
from mat3ra.prove import visualize_property

visualize_property(results.band_structure)
print(results.band_gap)